# 1. Download/Load SP500 stocks prices data.

In [1]:
!pip install statsmodels
!pip install matplotlib
!pip install pandas
!pip install numpy
!pip install pandas-datareader
!pip install yfinance
!pip install datetime
!pip install scikit-learn
!pip install PyPortfolioOpt
!pip install pandas-ta

In [3]:
# Reinstalar pandas-ta con una versión específica compatible con numpy
!pip uninstall -y pandas-ta
!pip install pandas-ta==0.3.14b0

Found existing installation: pandas_ta 0.3.14b0
Uninstalling pandas_ta-0.3.14b0:
  Successfully uninstalled pandas_ta-0.3.14b0
  Using cached pandas_ta-0.3.14b0-py3-none-any.whl


In [6]:
# Corregir el error de importación en pandas-ta
import os
import site

# Buscar el archivo directamente en site-packages
site_packages = site.getsitepackages()[0]
squeeze_pro_path = os.path.join(site_packages, 'pandas_ta', 'momentum', 'squeeze_pro.py')

# Verificar si el archivo existe
if os.path.exists(squeeze_pro_path):
    print(f"Archivo encontrado en: {squeeze_pro_path}")
    
    # Leer el contenido actual
    with open(squeeze_pro_path, 'r') as file:
        content = file.read()
    
    # Reemplazar 'from numpy import NaN as npNaN' por 'from numpy import nan as npNaN'
    if 'from numpy import NaN as npNaN' in content:
        content = content.replace('from numpy import NaN as npNaN', 'from numpy import nan as npNaN')
        
        # Guardar el archivo modificado
        with open(squeeze_pro_path, 'w') as file:
            file.write(content)
        print("El archivo ha sido corregido correctamente.")
    else:
        print("El archivo ya ha sido corregido o tiene un formato diferente.")
else:
    print(f"Buscando archivo en: {site_packages}")
    
    # Buscar el archivo en todo el site-packages
    found = False
    for root, dirs, files in os.walk(site_packages):
        if 'squeeze_pro.py' in files:
            found_path = os.path.join(root, 'squeeze_pro.py')
            print(f"Archivo encontrado en ubicación alternativa: {found_path}")
            
            # Leer el contenido
            with open(found_path, 'r') as file:
                content = file.read()
            
            if 'from numpy import NaN as npNaN' in content:
                content = content.replace('from numpy import NaN as npNaN', 'from numpy import nan as npNaN')
                
                # Guardar el archivo modificado
                with open(found_path, 'w') as file:
                    file.write(content)
                print("El archivo ha sido corregido correctamente.")
                found = True
                break
            else:
                print("El archivo ya ha sido corregido o tiene un formato diferente.")
                found = True
                break
    
    if not found:
        print("No se pudo encontrar el archivo 'squeeze_pro.py' en ninguna ubicación.")


Buscando archivo en: c:\Users\ASUS\Desktop\Juanes\Infraestructura\Proyecto-test1\.venv
Archivo encontrado en ubicación alternativa: c:\Users\ASUS\Desktop\Juanes\Infraestructura\Proyecto-test1\.venv\Lib\site-packages\pandas_ta\momentum\squeeze_pro.py
El archivo ha sido corregido correctamente.


In [ ]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

symbols_list = sp500['Symbol'].unique().tolist()

symbols_list

end_date = '2025-05-14'

start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end=end_date).stack()

df.index.names = ['date', 'ticker']

df.columns = df.columns.str.lower()

df


['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ADBE',
 'AMD',
 'AES',
 'AFL',
 'A',
 'APD',
 'ABNB',
 'AKAM',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'APO',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BAX',
 'BDX',
 'BRK-B',
 'BBY',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'BK',
 'BA',
 'BKNG',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF-B',
 'BLDR',
 'BG',
 'BXP',
 'CHRW',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'COR',
 'CNC',
 'CNP',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CAG',
 'COP',
 'ED',
 'STZ',
 'CEG',
 'COO',
